In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
from time import sleep

Go to LinkedIn and Login 

In [3]:
#go to LinkedIn

In [4]:
driver=webdriver.Chrome('./chromedriver_win32/chromedriver.exe')

In [5]:
url='https://www.linkedin.com/login'
driver.get(url)

In [6]:
#login

In [7]:
login_information=open('login.txt')
line=login_information.readlines()
username=line[0]
password=line[1]

In [8]:
#insert username 

In [9]:
user_field=driver.find_element_by_id('username')
user_field.send_keys(username)
sleep(3)

In [10]:
#inser password

In [11]:
password_field=driver.find_element_by_id('password')
password_field.send_keys(password)
sleep(2)

In [12]:
#click sign in

In [13]:
login_field=driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
login_field.click()
sleep(2)

search for Profile

In [14]:
#click search symbol

In [17]:
search_field=driver.find_element_by_xpath('//*[@id="global-nav-typeahead"]/input')


In [18]:
search_field.send_keys('Software Engineer people')

In [19]:
search_field.send_keys(Keys.RETURN)

In [20]:
people_field=driver.find_element_by_xpath('//*[@id="search-reusables__filters-bar"]/ul/li[1]')
people_field.click()

Scrape the URLs of the profiles

In [21]:
from bs4 import BeautifulSoup

In [22]:
def getURL(): 
    page_source=BeautifulSoup(driver.page_source)
    profiles=page_source.find_all('a',class_='app-aware-link')
    all_profile_URL=set()
    for profile in profiles:
        profile_ID=profile.get('href')
        
        all_profile_URL.add(profile_ID)
    return all_profile_URL

In [25]:
URLs_all_page=set()
for i in range(5):
    URLs_one_page=getURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    sleep(2)
    next_button=driver.find_element_by_class_name('artdeco-pagination__button--next')
    next_button.click()
    URLs_all_page.update(URLs_one_page)



write to a csv file

In [27]:
import csv

In [56]:
l=len(URLs_all_page)
for url in URLs_all_page:
    if 'ProfileUrn' in url:
        pass
    else:
        URLs_all_page.remove(url)


{'https://www.linkedin.com/in/neptuneboege?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABO_7igB3QgTNfE9BY2m8tJ8mgGngc7VKug', 'https://www.linkedin.com/in/andreibechet?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAN-WNoBpVyrODWH3AuT57HA30aVM4-Xy08', 'https://www.linkedin.com/in/igna-vermeulen?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABDnS-MBICo1846mQxRgrNVMd-LOCJJSjpw', 'https://www.linkedin.com/in/paul-gualotuna?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAA8S9vkBJ8HzvjSlxTdz9bkK2fHZEz-bBK4', 'https://www.linkedin.com/in/zehra-hay%C4%B1rc%C4%B1-39b4071b4?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADHk4WcBgD8Ghacvj6U1XNBqkCC9CCJtStQ', 'https://www.linkedin.com/in/sarra-hattab?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABNLK7UBMzmqbuUmU3ezn8ns3u-LESgSJNs', 'https://www.linkedin.com/in/corylevinson?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAmoE_ABVHI3X1e6Hn6ayQG63pU7GNAC510', 'https://www.linkedin.com/in/fongnancy?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACo

In [58]:
with open('output.csv', 'w',encoding='utf-8', newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)

        page_source = BeautifulSoup(driver.page_source, "html.parser")

        info_div = page_source.find('div',class_='display-flex justify-space-between pt2')
        info_name=info_div.find('h1')
        name=info_name.get_text().strip()
        print('--- Profile name is: ', name)
        info_occupation=info_div.find('div',class_="text-body-medium break-words")
        occupation=info_occupation.get_text().strip()
        print('--- Occupation: ', occupation)
        info_location=info_div.find('span', class_="text-body-small inline t-black--light break-words")
        location=info_location.get_text().strip()
        print('--- Profile location is: ', location)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:occupation, headers[3]:url})
        print('\n')

print('Mission Completed!')

- Accessing profile:  https://www.linkedin.com/in/neptuneboege?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABO_7igB3QgTNfE9BY2m8tJ8mgGngc7VKug
--- Profile name is:  Neptune Kaewsopa Böge
--- Occupation:  Software Engineer
--- Profile location is:  Dresden, Saxony, Germany


- Accessing profile:  https://www.linkedin.com/in/andreibechet?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAN-WNoBpVyrODWH3AuT57HA30aVM4-Xy08
--- Profile name is:  Andrei Bechet
--- Occupation:  Senior Software Engineer at ThoughtWorks
--- Profile location is:  Germany


- Accessing profile:  https://www.linkedin.com/in/igna-vermeulen?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABDnS-MBICo1846mQxRgrNVMd-LOCJJSjpw
--- Profile name is:  Igna Vermeulen
--- Occupation:  Software engineer - with a touch of data.
--- Profile location is:  Germany


- Accessing profile:  https://www.linkedin.com/in/paul-gualotuna?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAA8S9vkBJ8HzvjSlxTdz9bkK2fHZEz-bBK4
--- Profile name i

- Accessing profile:  https://www.linkedin.com/in/daria-iurchenko-254493112?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABww_qABp0vs4vQJ1Ln3DjKCX9j8ExNkJK0
--- Profile name is:  Daria Iurchenko
--- Occupation:  Software Engineer – PPRO
--- Profile location is:  Germany


- Accessing profile:  https://www.linkedin.com/in/youssef-fallaha?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAyYlUsBENNsH2y6xz3gmECZ0IJOdQ4g2A8
--- Profile name is:  Youssef Fallaha
--- Occupation:  Development Engineer Hardware-Software at engineering people GmbH
--- Profile location is:  Karlsruhe, Baden-Württemberg, Germany


- Accessing profile:  https://www.linkedin.com/in/rhys-howell?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABJa68YBZTe_BkjqDeJbYkWdY3RLSlOXnm4
--- Profile name is:  Rhys Howell
--- Occupation:  Software Engineer
--- Profile location is:  Germany


- Accessing profile:  https://www.linkedin.com/in/shu-lo-569634171?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACjZqSEBZVx7zCoN8AGeqAeui